In [51]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

train_ratio = 0.85
n_epochs = 50

horizon_pred = 1

# params = {
#     'seq_length' : [5, 15, 30],
#     'batch_size' : [20, 30, 50],
#     'units_layer1' : [50, 100, 200, 300],
#     'activation_layer1' : ['relu', 'sigmoid'],
#     'kernel_reg_factor' : [None, 0.01, 0.001, 0.0001], 
#     'dropout_layer1' : [None, 0.1, 0.2],
#     'dense_mid_size' : [None, 10, 20, 30],
#     'activation_output' : ['relu'],
#     'optim' : ['adam', 'SGD', 'RMSprop'],
#     'company_inputs' : [['PETR4.SA'], ['PETR4.SA', 'BZ=F', '6L=F']],
# }

params = {
    'seq_length' : [20, 30],
    'batch_size' : [20, 30],
    'units1' : [50, 100, 150, 200, 250, 300],
    'activation1' : ['relu'],
    'kernel_reg_factor1' : [0.001],
    'dropout_layer1' : [0.1],
    'dense_mid_size' : [50],
    'activation_out' : ['relu'],
    'optim' : ['nadam', 'RMSprop'],
    'company_inputs' : [['PETR4.SA']],
}

company_output = 'PETR4.SA'
end_date = '2024-10-30' 
start_date = '2020-01-01'


In [52]:
from sklearn.model_selection import ParameterGrid
from parametrized_training_model import parametrized_training

import mlflow
import mlflow.tensorflow

from sklearn.metrics import root_mean_squared_error, r2_score

mlflow.set_tracking_uri(uri="http://127.0.0.1:7777")
mlflow.set_experiment("LSTM Grid Search Test 9")

param_grid = ParameterGrid(params)
mlflow.tensorflow.autolog()

for param_sample in param_grid:
    with mlflow.start_run():

        mlflow.log_params(param_sample)
        mlflow.log_param("n_epochs", n_epochs)
        mlflow.log_param("train_ratio", train_ratio)
        mlflow.log_param("horizon_pred", horizon_pred)
        
        (scalerX, scalery, generator_train, generator_test, model) = parametrized_training(param_sample['company_inputs'], company_output, start_date, end_date, train_ratio,
                            horizon_pred, param_sample['seq_length'], param_sample['batch_size'],
                            param_sample['units1'], param_sample['activation1'], l2(param_sample['kernel_reg_factor1']), 
                            param_sample['dropout_layer1'], param_sample['dense_mid_size'], param_sample['activation_out'], param_sample['optim'])

        hist = model.fit(generator_train, epochs=n_epochs, validation_data=generator_test)

        X_gen_train, y_gen_train = generator_train[0]
        X_gen_test, y_gen_test = generator_test[0]
        y_pred_scaled = model.predict(X_gen_test)
        y_pred = scalery.inverse_transform(y_pred_scaled.reshape(-1,1))
        y_test = scalery.inverse_transform(y_gen_test.reshape(-1,1))
        # y_pred = y_pred_scaled.reshape(-1,1)
        # y_valid_f = y_gen_test.reshape(-1,1)

        mlflow.tensorflow.log_model(model, "model")

        # Calculate RMSE and R² score
        rmse = root_mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2-Score", r2)

        mlflow.end_run()


2024/11/17 21:39:15 INFO mlflow.tracking.fluent: Experiment with name 'LSTM Grid Search Test 9' does not exist. Creating a new experiment.


2024/11/17 21:39:15 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
[*********************100%***********************]  1 of 1 completed
2024/11/17 21:39:15 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.sequence.TimeseriesGenerator'>. Dataset logging skipped.
2024/11/17 21:39:15 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.sequence.TimeseriesGenerator'>. Dataset logging skipped.
c:\GitRepos\TC4-Thales\tech_challenge_4\training_model\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `

Epoch 1/50
49/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0592

51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0581 - val_loss: 0.0284
Epoch 2/50
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0077

51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0076 - val_loss: 0.0226
Epoch 3/50
45/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0047

...

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0020 - val_loss: 0.0012
Epoch 38/50
33/34 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0014

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 39/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0015 - val_loss: 0.0070
Epoch 40/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0014 - val_loss: 0.0063
Epoch 41/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0021 - val_loss: 0.0012
Epoch 42/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0024 - val_loss: 0.0085
Epoch 43/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 44/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0015 - val_loss: 0.0035
Epoch 45/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0019 - val_loss: 0.0041
Epoch 46/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0017 - val_loss: 0.0021
Epoch 47/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0019 - val_loss: 0.0015
Epoch 48/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0014 - val_loss: 0.0024
Epoch 49/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0020

2024/11/17 22:35:08 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: unsupported operand type(s) for *: 'int' and 'slice'
2024/11/17 22:35:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/17 22:35:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step


2024/11/17 22:35:17 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/17 22:35:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/17 22:35:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run burly-shrew-269 at: http://127.0.0.1:7777/#/experiments/199110122211212765/runs/17ad271112de4f2f863206b0e69b9b69.
2024/11/17 22:35:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:7777/#/experiments/199110122211212765.


In [53]:
# from sklearn.metrics import root_mean_squared_error, r2_score

# rmse = root_mean_squared_error(y_pred, y_valid_f)
# r2 = r2_score(y_pred, y_valid_f)

# print(f'RMSE: {rmse:.4f}')
# print(f'R² Score: {r2:.4f}')

In [54]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(range(len(y_gen_train)),y_gen_train, label="Training Data")
# plt.plot(range(len(y_gen_train),len(y_gen_train)+len(y_gen_test)),y_gen_test, label="Test Data")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [55]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(y_pred, label="Predicted Prices")
# plt.plot(y_valid_f, label="Real Prices")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [56]:
# import mlflow
# import mlflow.tensorflow

# mlflow.set_tracking_uri(uri="http://127.0.0.1:7777")
# mlflow.set_experiment("LSTM Experiments")

# from sklearn.metrics import root_mean_squared_error, r2_score

# mlflow.tensorflow.autolog()
# with mlflow.start_run():
#     mlflow.tensorflow.log_model(model, "model")

#     # Calculate RMSE and R² score
#     rmse = root_mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)

#     print(f'RMSE: {rmse:.4f}')
#     print(f'R² Score: {r2:.4f}')

#     mlflow.log_metric("RMSE", rmse)
#     mlflow.log_metric("R2-Score", r2)

In [57]:
# # Predict using the trained model
# future_steps = 10  # Number of days to predict
# future_data = data[-n_steps:].reshape(-1, n_steps, 1)
# predicted_prices = []

# for i in range(future_steps):
#     prediction = model.predict(future_data)[0, 0]
#     predicted_prices.append(prediction)
#     future_data = np.roll(future_data, -1, axis=1)
#     future_data[0, -1] = prediction

# # Inverse transform the predicted prices to original scale
# predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1)).flatten()

In [58]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(predicted_prices, label="Predicted Prices")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()